In [1]:
import numpy as np
import pandas as pd

In [7]:
df=pd.read_excel("РайфRates1.xlsx")
df.head(8)

,Tenor,Zero_Rate,Market_Rate
0,3 MO,2.16838,2.16838
1,6 MO,2.08842,2.00485
2,7 MO,1.97799,1.80419
3,8 MO,1.86053,1.54320
4,9 MO,1.76943,1.42687
5,10 MO,1.69402,1.35022
6,12 MO,1.63763,1.33825
7,2 YR,1.57774,1.58000


Сначала нам нужны коеффициенты дисконтирования на 3,6,9,12,15,18,21,24 месяца; для 15,18,21 их придется посчитать самим, исходя  из данных коеффициентов на 12 и 24 месяца

Только все ставки в df надо поделить на 100, потому что 2.16838 процентов =0.0216838

In [85]:
def get_discount_factors(myname):
    df=pd.read_excel(myname)
    D03=np.exp(-3/12*df[df['Tenor']=='3 MO']['Zero_Rate'].values[0]/100)
    D06=np.exp(-6/12*df[df['Tenor']=='6 MO']['Zero_Rate'].values[0]/100)
    D09=np.exp(-9/12*df[df['Tenor']=='9 MO']['Zero_Rate'].values[0]/100)
    D012=np.exp(-12/12*df[df['Tenor']=='12 MO']['Zero_Rate'].values[0]/100)
    D024=np.exp(-24/12*df[df['Tenor']=='2 YR']['Zero_Rate'].values[0]/100)
    D1224=D024/D012
    r1224=-np.log(D1224)/1
    D015=D012*np.exp(-r1224*3/12)
    D018=D012*np.exp(-r1224*6/12)
    D021=D012*np.exp(-r1224*9/12)
    mydict={}
    mydict[0/12]=1
    mydict[3/12]=D03
    mydict[6/12]=D06
    mydict[9/12]=D09
    mydict[12/12]=D012
    mydict[15/12]=D015
    mydict[18/12]=D018
    mydict[21/12]=D021
    mydict[24/12]=D024
    return mydict
mydict=get_discount_factors("РайфRates1.xlsx")
print(mydict)

{0.0: 1, 0.25: 0.9945937168347572, 0.5: 0.9896122294572238, 0.75: 0.9868169428372681, 1.0: 0.9837570626130552, 1.25: 0.980031147166131, 1.5: 0.9763193433799462, 1.75: 0.9726215978075101, 2.0: 0.9689378572042587}


In [112]:


def P(a,byears,mydict):
    if (a!=0):
        raise Exception
    return mydict[byears]

def F(t,Tyears,Syears,mydict):
#     print("T,S=",Tyears,Syears)
#     print(P(t,Tyears))
#     print(P(t,Syears))
#     print(P(t,Tyears)/P(t,Syears))
    return (P(t,Tyears,mydict)/P(t,Syears,mydict)-1)/(Syears-Tyears)

def d1(K,F,v):
    return (np.log(F/K)+v*v/2)/v

def d2(K,F,v):
    return (np.log(F/K)-v*v/2)/v

from scipy.stats import norm as normal_distrib
def BL(K,F,v,w):
    return F*w*normal_distrib.cdf(w*d1(K,F,v),0,1)-K*w*normal_distrib.cdf(w*d2(K,F,v),0,1)




def get_priceCap(N,K,sigm,Ts,myname):
    mydict=get_discount_factors(myname)
    otv=0
    for i,Ti in enumerate(Ts):
        if (i==0):
            continue
        vi=sigm*np.sqrt(Ts[i-1])
        tek_otv=P(0,Ti,mydict)*(Ti-Ts[i-1])*BL(K,F(0,Ts[i-1],Ti,mydict),vi,1)
        otv+=tek_otv
    return N*otv

def get_priceFloor(N,K,sigm,Ts,myname):
    mydict=get_discount_factors(myname)
    otv=0
    for i,Ti in enumerate(Ts):
        if (i==0):
            continue
        vi=sigm*np.sqrt(Ts[i-1])
        tek_otv=P(0,Ti,mydict)*(Ti-Ts[i-1])*BL(K,F(0,Ts[i-1],Ti,mydict),vi,-1)
        otv+=tek_otv
    return N*otv



### 1a) Interest rate floor. Strike = ATM%, Tenor = 2 years, quarterly payment. Notional = 1 000 000.

In [96]:
N=1000000
S=2/100 #спотовая ставка
K=S
get_priceFloor(N,K,0.25,[0/12,3/12,6/12,9/12,12/12,15/12,18/12,21/12,24/12],"РайфRates1.xlsx")

/Users/aleksandra/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: divide by zero encountered in double_scalars
  
/Users/aleksandra/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: divide by zero encountered in double_scalars


9368.378467357848

### 1b) Interest rate cap. Strike = ATM+1%, Tenor = 1 year, quarterly payment. Notional = 1 000 000.

In [97]:
N=1000000
S=2/100 #спотовая ставка
K=S*1.01
get_priceCap(N,K,0.25,[0/12,3/12,6/12,9/12,12/12,15/12,18/12,21/12,24/12],"РайфRates1.xlsx")

/Users/aleksandra/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: divide by zero encountered in double_scalars
  
/Users/aleksandra/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: divide by zero encountered in double_scalars


1066.9799968140453

### 2)Imagine that 3 months pass, and you are asked to reprice the contracts. Use file Rates2.xlsx and assume that volatility surface is flat at 20%.

In [100]:
mydict=get_discount_factors("РайфRates2.xlsx")
mydict

{0.0: 1,
 0.25: 1.0001395847410435,
 0.5: 1.000291992621549,
 0.75: 1.0004797400384613,
 1.0: 1.0003895758649746,
 1.25: 1.0001950940283653,
 1.5: 1.0000006500002112,
 1.75: 0.9998062437731626,
 2.0: 0.9996118753398707}

In [113]:
N=1000000
S=2/100 #спотовая ставка
K=S
get_priceFloor(N,K,0.2,[0/12,3/12,6/12,9/12,12/12,15/12,18/12,21/12,24/12],"РайфRates2.xlsx")

/Users/aleksandra/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in log
  
/Users/aleksandra/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in log


nan

In [114]:
N=1000000
S=2/100 #спотовая ставка
K=S*1.01
get_priceCap(N,K,0.25,[0/12,3/12,6/12,9/12,12/12,15/12,18/12,21/12,24/12],"РайфRates2.xlsx")

/Users/aleksandra/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in log
  
/Users/aleksandra/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in log


nan

Так получилось, потому что некоторые ставки стали отрицательные, F стало отрицательным, и np.log(F/K) стало неопределено

### 3)Try to give an example of product/optionality, which would be hedged with the instruments from the tasks 1 and 2.
Is it possible to have a cheaper hedge? Propose an instrument and explain your choice (no pricing needed here).

Cap может быть полезен, если мы имеем обязательства в плавающей ставке (зависящий от LIBOR) и хотим защититься от повышения этой плавающей ставки - тогда мы ставим например strike=2.5%, тогда если плавающая ставка выростет больше 2.5%, то нам наш контракт будет платить деньги  

Floor, наоборот, полезен, когда мы хотим защититься от понижения плавающей ставки